In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tanmay01bhatt
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench


100%|██████████| 3.51M/3.51M [00:00<00:00, 348MB/s]

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [4]:
data = pd.read_csv("/content/language-translation-englishfrench/eng_-french.csv")

In [5]:
data.head()

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !


In [7]:
data = data.sample(n=6000, random_state=42).reset_index(drop=True)
data.shape

(6000, 2)

In [8]:
data.isnull().sum()

,0
English words/sentences,0
French words/sentences,0


In [9]:
data.columns = ['source_text', 'target_text']

In [10]:
data["source_text"] = data["source_text"].astype(str)
data["target_text"] = data["target_text"].astype(str)

In [11]:
# Split dataset into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

In [12]:
train_dataset

Dataset({
    features: ['source_text', 'target_text', '__index_level_0__'],
    num_rows: 5400
})

# Tokenization

In [13]:
from transformers import AutoTokenizer

# Load a tokenizer for the chosen model (e.g., mT5 or mBART)
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [14]:
input_max_len = max([len(tokenizer.encode(text)) for text in data['source_text']])
input_max_len

43

In [15]:
output_max_len = max([len(tokenizer.encode(text)) for text in data['target_text']])
output_max_len

60

In [16]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['source_text'],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        examples['target_text'],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [19]:
# Set format for PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [20]:
train_dataset[0]

{'input_ids': tensor([250004,     87,  36663,     47,  51544,    398,    100,   2367,    398,
           6777,  18925,      5,      2,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1

# Model

In [21]:
import os

results_dir = "/content"
model_dir = "/content"

# Create the results directory if it doesn't exist
os.makedirs(results_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [22]:
from transformers import Seq2SeqTrainingArguments

from transformers import AutoModelForSeq2SeqLM

# AutoModelForSeq2SeqLM is a class in the Hugging Face Transformers library that automatically loads a pre-trained sequence-to-sequence model.
# It is used for tasks like machine translation, summarization, and other text generation tasks, where both the encoder and decoder are trained to process input and generate output sequences.


In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50")

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [24]:
# Set language-specific tokens if using mBART
model.config.decoder_start_token_id = tokenizer.lang_code_to_id["fr_XX"]  # French token
tokenizer.src_lang = "en_XX"
tokenizer.tgt_lang = "fr_XX"

In [25]:
# Seq2SeqTrainingArguments is a class in Hugging Face's Transformers library designed specifically for training sequence-to-sequence models.
# It provides various training configurations such as batch size, number of epochs, evaluation strategy, and output directory, optimized for tasks like translation, summarization, or text generation.

training_args = Seq2SeqTrainingArguments(
    output_dir=results_dir,
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    predict_with_generate=True,
    generation_max_length=128,
)


In [26]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

/tmp/ipython-input-4010235864.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanmay01bhatt (tanmay01bhatt-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


# Inference

In [ ]:
def translate_text(text):
    # Tokenize input
    inputs = loaded_tokenizer(text, return_tensors="pt", max_length=128, truncation=True)
    # Generate translation
    outputs = loaded_model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    # Decode the translation
    translation = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

# Example usage
text_to_translate = "The book of the generation of Jesus Christ"
translated_text = translate_text(text_to_translate)
print("Translated text:", translated_text)


In [ ]:
text_to_translate = "When you meet someone for the first time, be careful about your impressions"
translated_text = translate_text(text_to_translate)
print("Translated text:", translated_text)

# Save The Model

In [ ]:
# Create the directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)

# Save the fine-tuned model
trainer.save_model(model_dir)

# Save the tokenizer explicitly
tokenizer.save_pretrained(model_dir)


loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_dir)